# Weather patterns based on 
---
## Observations

---
## Set Up

In [1]:
# Dependencies
import pandas as pd
import requests
import time
import numpy as py
import matplotlib.pyplot as plt
import api_keys
from citipy import citipy

In [2]:
# location for save file
savefile = "Output/TBD.csv"

In [3]:
# Constants
lat_range = (-90, 90)
lon_range = (-180, 180)

---
### ID Locations for Testing

---
### Check Weather for Locations

---
### Clean Up Data and Save to File

---
### Data Visualization